<a href="https://colab.research.google.com/github/vedanshipathak/Mental-Health-Support-Chatbot/blob/main/Final_Model_mental_health_support.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain chromadb openai flask datasets evaluate torch transformers accelerate faiss-cpu


In [ ]:
import pandas as pd
import re
# Load the dataset into a pandas DataFrame
df = pd.read_csv(r'/content/counselchat-data.csv')

# Function to strip HTML tags (like <p>, <br>, etc.)
def remove_html_tags(text):
    clean = re.compile('<.*?>')
    return re.sub(clean, '', str(text))

# Apply cleaning to the 'answerText' column
df['clean_answer'] = df['answerText'].apply(remove_html_tags)

# Optional: Combine questionTitle and questionText for better context
df['full_question'] = df['questionTitle'].fillna('') + ' ' + df['questionText'].fillna('')

# Preview the cleaned data
df[['full_question', 'clean_answer']].head()
# Display first few rows
print(df.head())


                 questionID                                     questionTitle  \
0  5566fab2a64752d71ec3ca69  Escalating disagreements between mother and wife   
1  5566f94fa64752d71ec3ca64          I'm addicted to smoking. How can I stop?   
2  5567d26887a1cc0c3f3d8f46                    Keeping secrets from my family   
3  556bed15c969ba5861709df5         The Underlying Causes of Being Possessive   
4  556ba115c969ba5861709de6         Can I control anxiety without medication?   

                                        questionText  \
0  My wife and mother are having tense disagreeme...   
1  I'm planning to have baby, so I have to quit s...   
2  I have secrets in my mind, and I don't know wh...   
3  I am extremely possessive in my relationships ...   
4  I had a head injury a few years ago and my min...   

                                         questionUrl  \
0  https://counselchat.com/questions/escalating-d...   
1  https://counselchat.com/questions/i-m-addicted...   
2  https

In [ ]:
# Display column names
print("Column Names:", df.columns.tolist())


Column Names: ['questionID', 'questionTitle', 'questionText', 'questionUrl', 'topics', 'therapistName', 'therapistUrl', 'answerText', 'upvotes', 'clean_answer', 'full_question']


Sample Question: My wife and mother are having tense disagreements. In the past, they’ve had minor differences. For example, my wife would complain to me my mother is too overbearing; my mother would complain my wife is lazy.
However, it’s intensified lately. I think the cause is my wife talked back to her once. Now, any little disagreement is magnified, leading to major disagreements. What can I do?
Sample Answer: What you are describing is something psychologists have termed &#34;triangulation&#34; which is what happens when one family member will not talk to the one they have a problem with and goes to a third member of the family to complain instead. You have been &#34;triangulated&#34; by your wife and mother.This is often seen in families. It's seen everywhere. How many times have you had a problem with someone but you didn't go to them to tell them, you went to someone else to complain? It is usually difficult for a person to confront another, especially in relationships where t

In [ ]:
import html

# Apply to the cleaned answer column
df['decoded_answer'] = df['clean_answer'].apply(html.unescape)

# Optional: also decode the question column if needed
df['decoded_question'] = df['full_question'].apply(html.unescape)

# Preview
df[['decoded_question', 'decoded_answer']].head()

,decoded_question,decoded_answer
0,Escalating disagreements between mother and wi...,What you are describing is something psycholog...
1,I'm addicted to smoking. How can I stop? I'm p...,Hi. Good for you in planning ahead to do what'...
2,Keeping secrets from my family I have secrets ...,It sounds like keeping the secrets has become ...
3,The Underlying Causes of Being Possessive I am...,Hi there. It's great you are able to realize t...
4,Can I control anxiety without medication? I ha...,You didn't say what or how many medications yo...


In [ ]:
# Extract relevant columns
questions = df["decoded_question"].dropna().tolist()  # Remove NaN values if any
answers = df["decoded_answer"].dropna().tolist()

# Print a sample
print("Sample Question:", questions[0])
print("Sample Answer:", answers[0])

Sample Question: Escalating disagreements between mother and wife My wife and mother are having tense disagreements. In the past, they’ve had minor differences. For example, my wife would complain to me my mother is too overbearing; my mother would complain my wife is lazy.
However, it’s intensified lately. I think the cause is my wife talked back to her once. Now, any little disagreement is magnified, leading to major disagreements. What can I do?
Sample Answer: What you are describing is something psychologists have termed "triangulation" which is what happens when one family member will not talk to the one they have a problem with and goes to a third member of the family to complain instead. You have been "triangulated" by your wife and mother.This is often seen in families. It's seen everywhere. How many times have you had a problem with someone but you didn't go to them to tell them, you went to someone else to complain? It is usually difficult for a person to confront another, es

In [ ]:
!pip install langchain openai


In [ ]:
from langchain.prompts import PromptTemplate

# General Mental Health Support Prompt
general_prompt = PromptTemplate(
    template="""
You are a compassionate therapist. Respond with empathy, support, and actionable advice.
Maintain a warm and understanding tone. If necessary, suggest mindfulness, therapy, or self-care techniques.

Client: {client_message}
Therapist:
""",
    input_variables=["client_message"],
)

# Example usage:
client_message = "I've been feeling very anxious lately and don't know how to cope."
formatted_prompt = general_prompt.format(client_message=client_message)
print(formatted_prompt)



You are a compassionate therapist. Respond with empathy, support, and actionable advice.
Maintain a warm and understanding tone. If necessary, suggest mindfulness, therapy, or self-care techniques.

Client: I've been feeling very anxious lately and don't know how to cope.
Therapist:



In [ ]:
def get_prompt(client_message):
    if any(word in client_message.lower() for word in ["anxious", "panic", "overwhelmed"]):
        return anxiety_prompt.format(client_message=client_message)
    elif any(word in client_message.lower() for word in ["depressed", "hopeless", "sad"]):
        return depression_prompt.format(client_message=client_message)
    elif any(word in client_message.lower() for word in ["relationship", "partner", "marriage", "family"]):
        return relationship_prompt.format(client_message=client_message)
    else:
        return general_prompt.format(client_message=client_message)

# Example Test
test_message = "I'm having serious conflicts with my wife, and I feel stuck."
selected_prompt = get_prompt(test_message)
print(selected_prompt)



You are a compassionate therapist. Respond with empathy, support, and actionable advice.
Maintain a warm and understanding tone. If necessary, suggest mindfulness, therapy, or self-care techniques.

Client: I'm having serious conflicts with my wife, and I feel stuck.
Therapist:



In [ ]:
!pip install transformers torch accelerate


In [ ]:
from huggingface_hub import login
login()


In [ ]:
import requests

try:
    response = requests.get("https://huggingface.co", timeout=5)
    if response.status_code == 200:
        print("✅ Internet is working. Hugging Face is reachable!")
    else:
        print("⚠️ Internet issue: Received status code", response.status_code)
except requests.ConnectionError:
    print("❌ No internet connection detected!")


✅ Internet is working. Hugging Face is reachable!


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Load Mistral-7B model & tokenizer
model_name = "mistralai/Mistral-7B-Instruct-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")

print("Model loaded successfully!")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Model loaded successfully!


In [ ]:
def generate_response(client_message):
    # Format the structured prompt
    prompt = f"""
    You are a compassionate therapist. Respond with empathy, support, and actionable advice.
    Maintain a warm and understanding tone. If necessary, suggest mindfulness, therapy, or self-care techniques.

    Client: {client_message}
    Therapist:
    """

    # Tokenize input
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    # Generate response
    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=200, temperature=0.7, top_p=0.9)

    # Decode and return the response
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

# ✅ Test the chatbot
test_message = "I've been feeling very anxious lately and don't know how to cope."
response = generate_response(test_message)
print("\nChatbot Response:\n", response)


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Chatbot Response:
 
    You are a compassionate therapist. Respond with empathy, support, and actionable advice.
    Maintain a warm and understanding tone. If necessary, suggest mindfulness, therapy, or self-care techniques.

    Client: I've been feeling very anxious lately and don't know how to cope.
    Therapist:
    
    I'm sorry to hear that you're feeling anxious. It's completely normal to experience anxiety at times, but it's important to address it when it becomes overwhelming.
    
    One helpful technique to manage anxiety is mindfulness meditation. This involves focusing on the present moment and accepting your thoughts and feelings without judgment. You can start by finding a quiet place to sit or lie down, closing your eyes, and focusing on your breath. As you breathe in and out, try to let go of any distracting thoughts and just be present in the moment.
    


In [ ]:
!pip install datasets evaluate sacrebleu nltk


In [ ]:
!pip install rouge_score

In [ ]:
import evaluate
import math
import torch
import nltk
from transformers import AutoModelForCausalLM, AutoTokenizer

nltk.download("punkt")  # Needed for tokenization in BLEU & ROUGE

# Load BLEU and ROUGE metrics from Hugging Face's Evaluate library
bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")

# Function to calculate perplexity
def calculate_perplexity(response):
    inputs = tokenizer(response, return_tensors="pt").to("cuda")
    with torch.no_grad():
        outputs = model(**inputs, labels=inputs["input_ids"])
    loss = outputs.loss
    perplexity = math.exp(loss.item())
    return perplexity


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
def evaluate_response(generated_response, reference_response):
    # Compute BLEU score (expects full sentences, not tokenized words)
    bleu_score = bleu.compute(predictions=[generated_response], references=[[reference_response]])

    # Compute ROUGE score (expects full sentences)
    rouge_score = rouge.compute(predictions=[generated_response], references=[reference_response])

    # Compute perplexity
    perplexity = calculate_perplexity(generated_response)

    return {
        "BLEU Score": bleu_score["bleu"],
        "ROUGE Score": rouge_score["rouge1"],
        "Perplexity": perplexity
    }


In [ ]:
 import nltk
 nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [ ]:
# Example chatbot response vs reference therapist response
generated_response = """
I'm sorry to hear that you're feeling anxious. It's completely normal to experience anxiety at times, but it's important to address it when it becomes overwhelming.
One helpful technique to manage anxiety is mindfulness meditation. This involves focusing on the present moment and accepting your thoughts and feelings without judgment.
"""
reference_response = """
I hear that you're feeling anxious, and that's completely understandable. A helpful way to manage anxiety is practicing deep breathing exercises or grounding techniques.
Taking a moment to focus on your breath or journaling can also be beneficial. Would you like to explore more coping strategies together?
"""

# Run evaluation
scores = evaluate_response(generated_response, reference_response)
print(scores)


{'BLEU Score': 0.12811300935224262, 'ROUGE Score': np.float64(0.38834951456310685), 'Perplexity': 2.6409495669289966}


In [ ]:
!pip install -U sentence-transformers



In [ ]:
!pip install faiss-cpu langchain chromadb


In [ ]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# Load a free embedding model
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")  # Free alternative to OpenAIEmbeddings

# Example dataset
conversation_texts = [
    "How do I deal with anxiety?",
    "What should I do if I'm feeling very anxious?",
    "How can I manage stress effectively?",
    "I've been feeling overwhelmed at work, how do I handle it?"
]

# Convert questions to vector embeddings
vectors = embedding_model.encode(conversation_texts, convert_to_numpy=True)

# Create FAISS index
vector_array = np.array(vectors, dtype=np.float32)
faiss_index = faiss.IndexFlatL2(vector_array.shape[1])
faiss_index.add(vector_array)

print("✅ FAISS Index Created Successfully with SentenceTransformers!")



✅ FAISS Index Created Successfully with SentenceTransformers!


In [ ]:
def retrieve_context(query):
    """Find similar past questions from FAISS and return relevant context."""
    query_vector = np.array([embedding_model.encode(query)], dtype=np.float32)
    distances, indices = faiss_index.search(query_vector, k=2)  # Retrieve top 2 similar questions

    # Fetch similar past questions
    similar_texts = [conversation_texts[i] for i in indices[0]]
    return similar_texts

# ✅ Test retrieval
query = "I'm feeling really stressed lately. What should I do?"
retrieved_context = retrieve_context(query)
print("🔍 Retrieved Context:", retrieved_context)


🔍 Retrieved Context: ['How can I manage stress effectively?', "What should I do if I'm feeling very anxious?"]


In [ ]:
def generate_response_with_rag(client_message):
    # Retrieve similar past conversations
    context = retrieve_context(client_message)

    # Format prompt with retrieved knowledge
    prompt =  f"""
    You are a compassionate therapist. Respond with empathy, support, and actionable advice.
    Maintain a warm and understanding tone. If necessary, suggest mindfulness, therapy, or self-care techniques.

    Client: {client_message}
    Therapist:
    """

    # Tokenize input
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    # Generate response
    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=200, temperature=0.7, top_p=0.9)

    # Decode and return the response
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# ✅ Test the chatbot with RAG
test_message = "hello iam sad"
response = generate_response_with_rag(test_message)
print("\nChatbot Response with RAG:\n", response)


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Chatbot Response with RAG:
 
    You are a compassionate therapist. Respond with empathy, support, and actionable advice.
    Maintain a warm and understanding tone. If necessary, suggest mindfulness, therapy, or self-care techniques.

    Client: hello iam sad
    Therapist:
    
    Hello there. I'm sorry to hear that you're feeling sad. It's important to remember that it's okay to feel this way sometimes. Can you tell me more about what's been going on in your life that's been causing you to feel this way?


In [ ]:
def generate_response_with_rag(client_message):
    # Retrieve similar past conversations
    context = retrieve_context(client_message)

    # Format prompt with retrieved knowledge
    prompt =  f"""
    You are a compassionate therapist. Respond with empathy, support, and actionable advice.
    Maintain a warm and understanding tone. If necessary, suggest mindfulness, therapy, or self-care techniques.

    Relevant Context: {context}

    Client: {client_message}
    Therapist:
    """

    # Tokenize input
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    # Generate response
    with torch.no_grad():
        outputs = model.generate(**inputs,max_new_tokens=200, temperature=0.7, top_p=0.9,eos_token_id=tokenizer.eos_token_id)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Split at 'Therapist:' and return only the actual answer
    response = response.split("Therapist:")[-1].strip()
    return response


In [ ]:
test_messages = [
    "hello i am sad",
    "i feel overwhelmed by everything at work",
    "my mom and wife keep arguing and it’s stressing me out",
    "i can't sleep at night and keep overthinking"
]

for msg in test_messages:
    print("\nClient:", msg)
    print("Therapist:", generate_response_with_rag(msg))


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Client: hello i am sad


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Therapist: Hello, I'm sorry to hear that you're feeling sad. It's important to remember that it's okay to feel this way and that you're not alone. Can you tell me more about what's been going on and how you're feeling?

Client: i feel overwhelmed by everything at work


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Therapist: I'm sorry to hear that you're feeling overwhelmed at work. It sounds like things are becoming too much to handle. It's important to remember that it's okay to feel overwhelmed, and that there are ways to manage these feelings.

    One helpful technique is to practice mindfulness. This means being present in the moment and focusing on your thoughts and feelings without judgment. You can try taking a few deep breaths and focusing on your breath, or simply observing your thoughts and emotions without getting caught up in them.

    Another helpful strategy is to prioritize your tasks. Make a list of everything you need to do, and then rank them in order of importance. This can help you focus on the most pressing tasks first and prevent yourself from feeling overwhelmed.

    It's also important to take care of yourself outside of work. Make sure you're getting enough sleep, eating well, and engaging in regular physical activity. These self

Client: my mom and wife keep arguing

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Therapist: I'm sorry to hear that you're feeling stressed due to the arguments between your mom and wife. It's understandable that this can be a difficult situation to navigate, especially when it involves people you love.

    It's important to remember that everyone has their own unique perspective and way of communicating. It's possible that there may be misunderstandings or unspoken feelings that are contributing to the arguments.

    One approach you could try is to encourage both your mom and wife to sit down and have a conversation together. This can give them a chance to express their thoughts and feelings in a calm and respectful manner. It may also help them to better understand each other's perspectives and find common ground.

    If this doesn't work, it may be helpful to seek the support of a therapist who can help facilitate the conversation and provide guidance on effective communication skills.

    In the meantime, it'

Client: i can't sleep at night and keep overthi

Backend Setup


In [ ]:

!pip install flask flask-cors --quiet
!pip install pyngrok --quiet
!pip install transformers accelerate --quiet


In [ ]:
from pyngrok import ngrok
# Set your authtoken here (paste your token inside the quotes)
ngrok.set_auth_token("NGROK_TOKEN")


In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS
from pyngrok import ngrok
from langchain.prompts import PromptTemplate
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch


general_prompt = PromptTemplate(
    template="""
You are a compassionate therapist. Respond with empathy, support, and actionable advice.
Maintain a warm and understanding tone. If necessary, suggest mindfulness, therapy, or self-care techniques.

Client: {client_message}
Therapist:""",
    input_variables=["client_message"],
)

anxiety_prompt = PromptTemplate(
    template="""
You are a calm and reassuring mental health assistant. The user is feeling anxious or overwhelmed.
Offer calming techniques, grounding exercises, and positive affirmations.

Client: {client_message}
Therapist:""",
    input_variables=["client_message"],
)

depression_prompt = PromptTemplate(
    template="""
You are a caring and understanding therapist. The user feels depressed, hopeless, or sad.
Listen supportively and offer encouragement, self-compassion practices, and gentle help.

Client: {client_message}
Therapist:""",
    input_variables=["client_message"],
)

relationship_prompt = PromptTemplate(
    template="""
You are a relationship counselor. The user is experiencing issues in relationships or family.
Help them understand their emotions and suggest communication strategies.

Client: {client_message}
Therapist:""",
    input_variables=["client_message"],
)

def get_prompt(client_message):
    message = client_message.lower()
    if any(word in message for word in ["anxious", "panic", "overwhelmed"]):
        return anxiety_prompt.format(client_message=client_message)
    elif any(word in message for word in ["depressed", "hopeless", "sad"]):
        return depression_prompt.format(client_message=client_message)
    elif any(word in message for word in ["relationship", "partner", "marriage", "family"]):
        return relationship_prompt.format(client_message=client_message)
    else:
        return general_prompt.format(client_message=client_message)

def run_model(prompt):
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(**inputs, max_new_tokens=150)
    return tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

app = Flask(__name__)
CORS(app)

@app.route("/chat", methods=["POST"])
def chat():
    data = request.json
    user_input = data.get("message", "")
    prompt = get_prompt(user_input)
    response = run_model(prompt)
    return jsonify({"response": response})

port = 5001
public_url = ngrok.connect(port)
print(f"🚀 Public URL: {public_url}")
app.run(port=port)
